In [1]:
from __future__ import print_function
import sys
from spotify_client import SpotifyClient
sys.path.append('../gexf')
#from gexf import Gexf, GexfImport
import networkx as nx
import numpy as np
import time
from random import randint
import json
import pandas as pd


In [2]:
labels=['artistId','name', 'relatedArtists', 'followers', 'popularity','genres']
artist_df = pd.DataFrame.from_records([], columns=labels)
#artist_df.head()

In [3]:
################################
# general variables
################################
jumping_probability = 0.80

################################
class ArtistGraph():
    def __init__(self):
        self.sc = SpotifyClient()
        self.graph_nx = nx.DiGraph() #networkx graph for storing data
        #self.gexf = Gexf("creator_name","music graph") #gexf graph format for plotting
        #self.graph_gexf = self.gexf.addGraph("directed","unweighted","time")
    
   
    def pick_next_artist(self,previous_artist):

             
        related_artist_list = previous_artist.related
        probability_pick = np.random.random()
        
        if probability_pick < jumping_probability:
            next_artist = np.random.choice(related_artist_list)
        else:
            next_artist= np.random.choice(initial_artist_list)
            
        return next_artist

    def walk_for_seed(self, seed_artist):
        global artist_df,artist_list
        G = self.graph_nx #create an ampty graph       
        start_time = time.time()
        previous_artist= ag.sc.get_artist_by_id(seed_artist)
        #artist_list=[] 
        i=0
                        
        while (len(artist_df) < 201):
            try:
                #print("seed_artist")
                if previous_artist.id not in artist_list:
                #if previous_artist.name not in artist_df['name']:
                    G.add_node(previous_artist.id , label = previous_artist.name)
                    artist_list.append(previous_artist.id)
                    artist_df=artist_df.append([{"artistId":previous_artist.id,"name":previous_artist.name,
                            "relatedArtists":previous_artist.related,"followers":previous_artist.followers,
                                                 "popularity":previous_artist.popularity,"genres":previous_artist.genres}], ignore_index=True)
                
                    #print("previous:",previous_artist.name)
                    
                next_artist_id = self.pick_next_artist(previous_artist)
                next_artist= self.sc.get_artist_by_id(next_artist_id)
                #G.add_node(next_artist.id , label = next_artist.name)
                
                #artist_list.append(next_artist.id)
                #print("next:",next_artist.name)
                if next_artist_id in previous_artist.related:
                    G.add_edge(previous_artist.id,next_artist.id , label= previous_artist.name+'-'+next_artist.name)
                previous_artist= next_artist  
                
                #break             
                       
                #print("artist in list:",len(artist_list))                        
                
                
            except ValueError:
                print("exception!")
                #break
            i=i+1          
            if i%50 ==0:
                print(artist_df.shape)
                print(int(time.time() - start_time))               
                time.sleep(10)
                
        
          
        nx.write_gexf(G, "rw.gexf") 
        print("num of nodes:",len(G))
        print("num od edges:",G.number_of_edges())
        with open('artists_list.txt', 'w') as f:
            for artist in artist_list:
                f.write(artist + '\n')
        
    
        
        
       
    



    

    


In [4]:

ag = ArtistGraph()
artist_list = [] #the artist that we crawl 
with open('seed_id.txt') as f:
    initial_artist_list = f.read().splitlines()   
  
    
r=randint(1, 70)
start_artist = initial_artist_list[r-1]  
    
    #seed_artist = ag.sc.get_artist_by_id(start_artist)    
    
ag.walk_for_seed(start_artist)
       
        

exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
(28, 6)
6
exception!
exception!
exception!
exception!
exception!
exception!
exception!
(48, 6)
24
(65, 6)
43
(85, 6)
62
(96, 6)
81
(108, 6)
100
(113, 6)
119
(119, 6)
137
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
(124, 6)
154
(131, 6)
173
(135, 6)
192
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
exception!
(135, 6)
209
(137, 6)
228
(137, 6)
247
(141, 6)
265
(142, 6)
284
(142, 6)
303
(142, 6)
322
(142, 6)
341
exception!
(146, 6)
359
(146, 6)
378
(147, 6)
397
(147, 6)
415
(148, 6)
434
(150, 6)
452
(150, 6)
471
(150, 6)
490
(150, 6)
510
(150, 6)
529
(152, 6)
548
(152, 6)
568
(152, 6)
587
(152, 6)
606
(152, 6)
625
(153, 6)
644
(153, 6)
662
(153

In [5]:
artist_df.head()

,artistId,name,relatedArtists,followers,popularity,genres
0,5FTtKfa7DciDLMolfVdMMP,Haydeh,[],782,15,[]
1,05owR1ds02eaZgCrtfATRF,Fereshteh,"[7DCmQW885aZQY5fhN1C2PD, 7MYldHzLrKMxUl7AcNTmH...",514,10,[classic persian pop]
2,7MYldHzLrKMxUl7AcNTmHO,Giti,"[7DCmQW885aZQY5fhN1C2PD, 05owR1ds02eaZgCrtfATR...",678,14,[classic persian pop]
3,4DKaZIByJZluOmixSYIjjk,Siavash Ghomayshi,"[2jvB5M73tBZlWybPmxIaw7, 1xA7r3Y1UGCp3rQMkgFXz...",26353,39,[classic persian pop]
4,2jvB5M73tBZlWybPmxIaw7,Dariush,"[5b1CDxqOGnXr5M1DUn2XQh, 5WCHc16VvzVCmBKH70CYf...",21990,39,[classic persian pop]


In [6]:
artist_df.shape

(201, 6)

In [7]:
artist_df.to_csv("artists_df.csv")